In [1]:
from itertools import product
import pygame
import os
from itertools import permutations

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
A=[[0,0,0,0,4],
   [0,0,3,0,0],
   [3,2,0,5,0],
   [0,0,1,0,0],
   [0,0,0,0,4]]

A2=[[0,0,0,0,5,0,0,0,0],
   [0,0,0,4,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0],
   [0,0,0,0,1,0,0,0,0],
   [0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,8,0,0,0],
   [0,0,0,0,9,0,0,0,0]]

CELL = 30
MARGIN = 2
BG_COLOR = (255, 255, 255)
TEXT_COLOR = (0, 0, 0)
numeros = [
    [0,4,9],[1,3,8],[0,0,1],[0,-4,5],[-1,-3,4]
]

Matrice=A2
combinations = [[a,b,c,d,e,f,g,h,i] for a,b,c,d,e,f,g,h,i in product(range(1,5), repeat=9)]


In [3]:
print(len(combinations))
print(4**9)

262144
262144


In [4]:
element = [3,3,3,1,2,1,1,4,1]
element in combinations


True

In [5]:
### Définition de fonctions ###
def extraction_matrice(matrice, forme):
    n = len(matrice)
    if forme == 1:
        # Enlever 1ère ligne et 1ère colonne
        return [row[1:] for row in matrice[1:]]
    elif forme == 2:
        # Enlever 1ère ligne et dernière colonne
        return [row[:-1] for row in matrice[1:]]
    elif forme == 3:
        # Enlever dernière ligne et dernière colonne
        return [row[:-1] for row in matrice[:-1]]
    elif forme == 4:
        # Enlever 1ère colonne et dernière ligne
        return [row[1:] for row in matrice[:-1]]
    return []


def test(matrice,forme,chemin):
    n = len(matrice)
    L=forme[0]
    forme=forme[1:]

    positions_haut_gauche=[[k,0] for k in range(n)]+[[0,k] for k in range(1,n)]
    positions_bas_droite=[[k,n-1] for k in range(n)]+[[n-1,k] for k in range(1,n)]
    positions_haut_droite=[[k,n-1] for k in range(n)]+[[0,k] for k in range(0,n-1)]
    positions_bas_gauche=[[k,0] for k in range(n)]+[[n-1,k] for k in range(1,n)]
    positions = [positions_haut_gauche,positions_haut_droite,positions_bas_droite,positions_bas_gauche]
    
    pos=positions[L-1]
    nbre = [matrice[p][q] for (p, q) in pos]
    #print(nbre)
    valeurs = sorted(set(nbre))      # unique + tri
    if valeurs and valeurs[0] == 0:  # si le premier est zéro
        valeurs.pop(0)
    #print(valeurs)
    
    if len(valeurs) == 1:  # contrainte respectée
        val = valeurs[0]
        # Ajouter au chemin et continuer récursivement avec la matrice réduite
        sous_matrice = extraction_matrice(matrice,L)
        chemin = chemin + [val]
        if n==2:
            if val==2:
                sous_matrice = extraction_matrice(matrice,L)
                return test(sous_matrice,forme,chemin)
            return "erreur"
        elif n==1:
            if val==1:
                return chemin
            return "erreur"
        else:
            return test(sous_matrice,forme,chemin)
    
    if len(valeurs) == 0:
        val=-1
        sous_matrice = extraction_matrice(matrice,L)
        chemin = chemin + [val]
        if n==1:
            return chemin
        return test(sous_matrice,forme,chemin)

    
    return "erreur"


### fonctions d'affichage ###

############
import pygame
import os

# --- Constantes ---
CELL = 40
MARGIN = 2
BG_COLOR = (240, 240, 240)
TEXT_COLOR = (0, 0, 0)

# Couleurs fixes pour les numéros 1 à 9
NUM_COLORS = {
    1: (255, 0, 0),       # rouge
    2: (0, 128, 0),       # vert
    3: (0, 0, 255),       # bleu
    4: (255, 165, 0),     # orange
    5: (128, 0, 128),     # violet
    6: (0, 200, 200),     # cyan
    7: (255, 192, 203),   # rose
    8: (139, 69, 19),     # marron
    9: (128, 128, 128),   # gris
}

def dessiner_carre(surface, coord, couleur, dx=0, dy=0, numero=None, font=None):
    """Dessine une cellule carrée et éventuellement un numéro"""
    x, y = coord
    rect_pixel = pygame.Rect((x+dx)*CELL, (y+dy)*CELL, CELL-MARGIN, CELL-MARGIN)
    pygame.draw.rect(surface, couleur, rect_pixel)
    pygame.draw.rect(surface, (0,0,0), rect_pixel, 1)

    if numero is not None and font is not None:
        text_surf = font.render(str(numero), True, TEXT_COLOR)
        text_rect = text_surf.get_rect(center=rect_pixel.center)
        surface.blit(text_surf, text_rect)

def dessiner_L_colle(surface, rect, forme, couleur, dx=0, dy=0):
    xmin, ymin, xmax, ymax = rect
    coords = []

    if forme == 1:
        coords += [(x, ymin-1) for x in range(xmin-1, xmax+1)]
        coords += [(xmin-1, y) for y in range(ymin, ymax+1)]
        new_xmin = xmin-1
        new_ymin = ymin-1
        new_xmax = xmax
        new_ymax = ymax
    elif forme == 2:
        coords += [(x, ymin-1) for x in range(xmin, xmax+2)]
        coords += [(xmax+1, y) for y in range(ymin, ymax+1)]
        new_xmin = xmin
        new_ymin = ymin-1
        new_xmax = xmax+1
        new_ymax = ymax
    elif forme == 3:
        coords += [(x, ymax+1) for x in range(xmin, xmax+2)]
        coords += [(xmax+1, y) for y in range(ymin, ymax+2)]
        new_xmin = xmin
        new_ymin = ymin
        new_xmax = xmax+1
        new_ymax = ymax+1
    elif forme == 4:
        coords += [(x, ymax+1) for x in range(xmin-1, xmax+1)]
        coords += [(xmin-1, y) for y in range(ymin, ymax+2)]
        new_xmin = xmin-1
        new_ymin = ymin
        new_xmax = xmax
        new_ymax = ymax+1

    if surface is not None:
        for (x, y) in coords:
            rect_pixel = pygame.Rect((x+dx)*CELL, (y+dy)*CELL, CELL-MARGIN, CELL-MARGIN)
            pygame.draw.rect(surface, couleur, rect_pixel)
            pygame.draw.rect(surface, (0,0,0), rect_pixel, 1)

    return (new_xmin, new_ymin, new_xmax, new_ymax)

def enregistrer_sequence(sequence, chemin, nom_fichier="image.png", numeros=None):
    """
    sequence : liste des formes L [1,2,3,...]
    chemin : liste associant chaque L à un numéro
    numeros : liste de [x, y, nbre] à afficher
    """
    n = len(sequence)
    pygame.init()
    font = pygame.font.SysFont(None, CELL-5)

    # Carré central
    cx = cy = 0
    rect = (cx, cy, cx, cy)

    # Calculer rectangle final
    temp_rect = rect
    for forme in reversed(sequence[:-1]):
        temp_rect = dessiner_L_colle(None, temp_rect, forme, couleur=None)

    xmin, ymin, xmax, ymax = temp_rect
    xmin = min(xmin, cx)
    ymin = min(ymin, cy)
    xmax = max(xmax, cx)
    ymax = max(ymax, cy)

    dx = -xmin
    dy = -ymin

    largeur = (xmax - xmin + 1) * CELL
    hauteur = (ymax - ymin + 1) * CELL

    # Ajout d'une zone en bas pour la légende (9 cases de haut max)
    hauteur_legende = CELL * 2
    surface = pygame.Surface((largeur, hauteur + hauteur_legende))
    surface.fill(BG_COLOR)

    # --- Dessin ---
    # Dessiner carré central (lié au dernier numéro de chemin)
    couleur_centre = NUM_COLORS[chemin[-1]]
    dessiner_carre(surface, (cx, cy), couleur_centre, dx=dx, dy=dy)

    # Dessiner L concentriques avec les couleurs liées aux numéros
    temp_rect = (cx, cy, cx, cy)
    for i, forme in enumerate(reversed(sequence[:-1]), start=1):
        numero_associe = chemin[-1-i]  # remonter correctement
        couleur = NUM_COLORS[numero_associe]
        temp_rect = dessiner_L_colle(surface, temp_rect, forme, couleur, dx=dx, dy=dy)

    # Ajouter les numéros connus
    if numeros is not None:
        for x, y, val in numeros:
            dessiner_carre(surface, (x+dx, y+dy), (200,200,200), numero=val, font=font)

    # --- Légende ---
    legende_y = hauteur+20  # démarre sous le carré
    for i in range(1, 10):
        couleur = NUM_COLORS[i]
        rect_pixel = pygame.Rect((i-1)*CELL, legende_y, CELL-MARGIN, CELL-MARGIN)
        pygame.draw.rect(surface, couleur, rect_pixel)
        pygame.draw.rect(surface, (0,0,0), rect_pixel, 1)

        text_surf = font.render(str(i), True, TEXT_COLOR)
        text_rect = text_surf.get_rect(center=rect_pixel.center)
        surface.blit(text_surf, text_rect)

    # Sauvegarde
    os.makedirs("Image", exist_ok=True)
    chemin_fichier = os.path.join("Image", nom_fichier)
    pygame.image.save(surface, chemin_fichier)
    print(f"Image enregistrée : {chemin_fichier}")
    pygame.quit()


def enregistrer_sequence_matrice(sequence, chemin, matrice, nom_fichier="image_matrice.png",numeros=None):
    """
    Dessine une séquence de L + numéros de la matrice complète.
    - sequence : liste des formes L [1,2,3,...]
    - chemin   : liste associant chaque L à un numéro
    - matrice  : liste de listes (matrice complète avec 0 = vide)
    """
    pygame.init()
    font = pygame.font.SysFont(None, CELL-5)

    # Carré central
    cx = cy = 0
    rect = (cx, cy, cx, cy)

    # Calculer rectangle final (comme avant)
    temp_rect = rect
    for forme in reversed(sequence[:-1]):
        temp_rect = dessiner_L_colle(None, temp_rect, forme, couleur=None)

    xmin, ymin, xmax, ymax = temp_rect
    xmin = min(xmin, cx)
    ymin = min(ymin, cy)
    xmax = max(xmax, cx)
    ymax = max(ymax, cy)

    dx = -xmin
    dy = -ymin

    largeur = (xmax - xmin + 1) * CELL
    hauteur = (ymax - ymin + 1) * CELL

    # Ajout zone légende
    hauteur_legende = CELL * 2
    surface = pygame.Surface((largeur, hauteur + hauteur_legende))
    surface.fill(BG_COLOR)

    # --- Dessin ---
    # Carré central
    couleur_centre = NUM_COLORS[chemin[-1]]
    dessiner_carre(surface, (cx, cy), couleur_centre, dx=dx, dy=dy)

    # L concentriques
    temp_rect = (cx, cy, cx, cy)
    for i, forme in enumerate(reversed(sequence[:-1]), start=1):
        numero_associe = chemin[-1-i]
        couleur = NUM_COLORS[numero_associe]
        temp_rect = dessiner_L_colle(surface, temp_rect, forme, couleur, dx=dx, dy=dy)

    # --- Ajouter les numéros de la matrice ---
    nb_lignes = len(matrice)
    nb_cols = len(matrice[0]) if nb_lignes > 0 else 0

    # ⚠️ Ici : on aligne la matrice en partant de xmin, ymin
    for j in range(nb_lignes):      # ligne = y
        for i in range(nb_cols):    # colonne = x
            val = matrice[j][i]
            if val != 0:
                # position réelle dans le repère L
                x = xmin + i
                y = ymin + j
                dessiner_carre(surface, (x, y), (220, 220, 220), dx=dx, dy=dy, numero=val, font=font)

    # --- Légende ---
    legende_y = hauteur + 20
    for i in range(1, 10):
        couleur = NUM_COLORS[i]
        rect_pixel = pygame.Rect((i-1)*CELL, legende_y, CELL-MARGIN, CELL-MARGIN)
        pygame.draw.rect(surface, couleur, rect_pixel)
        pygame.draw.rect(surface, (0,0,0), rect_pixel, 1)

        text_surf = font.render(str(i), True, TEXT_COLOR)
        text_rect = text_surf.get_rect(center=rect_pixel.center)
        surface.blit(text_surf, text_rect)
    
    # Ajouter les numéros connus
    if numeros is not None:
        for x, y, val in numeros:
            dessiner_carre(surface, (x+dx, y+dy), (200,200,200), numero=val, font=font)

    # Sauvegarde
    os.makedirs("Image", exist_ok=True)
    chemin_fichier = os.path.join("Image", nom_fichier)
    pygame.image.save(surface, chemin_fichier)
    print(f"Image enregistrée : {chemin_fichier}")
    pygame.quit()


def enregistrement_des_grilles(resultat,nbre):
    a=resultat[0]
    b=resultat[1]
    for k in range(2,len(resultat)):
        c=resultat[k]
        enregistrer_sequence_matrice(a, b, c, nom_fichier=f"image_matrice_{nbre}_{k}.png",numeros=numeros)

In [6]:

#print(combinations[0])
Liste=[]
for Forme in combinations:
    chemin=test(Matrice,Forme,[])
    if chemin!="erreur":
        Liste.append([Forme,chemin])


Global = Liste
Global_updaté = []

for [f,c] in Global:
    if c[-1]==1:
        Global_updaté.append([f,c])

Global_updatée_bis = []

for [f,c] in Global_updaté:
    if f[-1]==1:
        Global_updatée_bis.append([f,c])

Global_updatée_2 = []

# limites max autorisées pour chaque valeur
limites = {9: 4, 8: 4, 7: 5, 6: 5, 5: 6, 4: 6, 3: 7, 2: 7}

for [f, c] in Global_updatée_bis:
    indices = {x: (c.index(x) if x in c else None) for x in range(2, 10)}

    # Vérifie uniquement les indices présents (pas None)
    conditions_ok = all(
        (indices[val] is None) or (indices[val] <= limite)
        for val, limite in limites.items()
    )

    if conditions_ok:
        Global_updatée_2.append([f, c])


Global_updatée_3 = []

for [f,c] in Global_updatée_2:
    c[-2]=2
    Global_updatée_3.append([f,c])


#Liste_unique_updatée_4 = []
#Global_updatée_4 = []

#for [f,c] in Global_updatée_3:
 #   if ((c[5]==-1) and (c[6]==-1)):
  #      Liste_unique_updatée_4.append(c)
   #     Global_updatée_4.append([f,c])


Global_updatée_5 = []

for [f,c] in Global_updatée_3:
    if ((f[-2]==3) or (f[-2]==4)):
        Global_updatée_5.append([f,c])

Global_updatée_6 = []
possibilités = [[6,3,7],[6,7,3],[3,6,7],[3,7,6],[7,6,3],[7,3,6]]

for [f, c] in Global_updatée_5:
    for k in possibilités:
        c_copy = c[:]   # copie pour ne pas modifier l’original
        idx = 0         # index dans la possibilité
        for i, val in enumerate(c_copy):
            if val == -1:        # si on a un trou à remplir
                c_copy[i] = k[idx]
                idx += 1
        Global_updatée_6.append([f, c_copy])




In [7]:
# #C=test(Matrice, element, [])
# #print(C)
# globall = [[3, 3, 3, 1, 2, 1, 1, 4, 1], [9, 8, -1, 5, 4, -1, -1, 2, 1]]
# print(globall)


# print(Global_updaté[0])

# print(globall in Liste)
# print(globall in Global)
# print(globall in Global_updaté)
# print(globall in Global_updatée_2)
# print(globall in Global_updatée_3)

# print(globall in Global_updatée_5)


In [8]:
# print(len(Global_updatée_5))
# sum=0
# for [Forme,Chemin] in Global_updatée_5:
#     sum+=1
#     print([Forme,Chemin])
#     enregistrer_sequence(Forme, f"puzzle_{sum}.png",numeros=numeros)

In [9]:

def construire_carre_emboite(orientations, valeurs):
    taille_max = 9
    matrice = [[0 for _ in range(taille_max)] for _ in range(taille_max)]
    
    x0, y0 = 0, 0  # Coin supérieur gauche du carré courant
    taille = taille_max
    
    for i, orientation in enumerate(orientations):
        valeur = valeurs[i]  # Valeur à placer
        
        if orientation == 1:
            # Angle en haut à gauche → haut + gauche
            for j in range(taille):
                matrice[y0][x0 + j] = valeur  # Ligne du haut
                matrice[y0 + j][x0] = valeur  # Colonne de gauche
            # Nouveau carré en haut à gauche
            x0, y0 = x0 + 1, y0 + 1

        elif orientation == 2:
            # Angle en haut à droite → haut + droite
            for j in range(taille):
                matrice[y0][x0 + j] = valeur  # Ligne du haut
                matrice[y0 + j][x0 + taille - 1] = valeur  # Colonne de droite
            # Nouveau carré en haut à droite
            x0, y0 = x0, y0 + 1

        elif orientation == 3:
            # Angle en bas à droite → bas + droite
            for j in range(taille):
                matrice[y0 + taille - 1][x0 + j] = valeur  # Ligne du bas
                matrice[y0 + j][x0 + taille - 1] = valeur  # Colonne de droite
            # Nouveau carré en bas à droite
            x0, y0 = x0, y0

        elif orientation == 4:
            # Angle en bas à gauche → bas + gauche
            for j in range(taille):
                matrice[y0 + taille - 1][x0 + j] = valeur  # Ligne du bas
                matrice[y0 + j][x0] = valeur  # Colonne de gauche
            # Nouveau carré en bas à gauche
            x0, y0 = x0 + 1, y0

        taille -= 1  # Le carré suivant est plus petit

    return matrice

# test = construire_carre_emboite([1,1,1,1,1,3,1,1,1], [i for i in range(1,10)])
# print(test)

contraintes = [[-1, -1, -1, -1, -1, -1, 7, -1, -1], [-1, -1, -1, -1, -1, 2, -1, -1, -1], [-1, -1, 3, -1, -1, -1, -1, -1, -1], [-1, -1, -1, 6, -1, -1, -1, -1, -1]]

def contraintes_L_C(L_C, contraintes):
    L = L_C[0]
    C = L_C[1]
    matrice = construire_carre_emboite(L, C)
    for side, contrainte in enumerate(contraintes):
        if side == 0 or side == 2:
            for col in range(len(contrainte)):
                if contrainte[col] != -1:
                    if contrainte[col] not in [ligne[col] for ligne in matrice]:
                        return False
        elif side == 1 or side == 3:
            for line in range(len(contrainte)):
                if contrainte[line] != -1:
                    if contrainte[line] not in matrice[line]:
                        return False
    return True


globall = [[3, 3, 3, 1, 2, 1, 1, 4, 1], [9, 8, 7, 5, 4, 6, 3, 2, 1]]
exemple = construire_carre_emboite(globall[0], globall[1])
contraintes_L_C(globall, contraintes)

Global_updatée_7 = []

for [f,c] in Global_updatée_6:
    if contraintes_L_C([f,c], contraintes):
        Global_updatée_7.append([f,c])

print(len(Global_updatée_7))

Global_updatée_8 = []

for [f, c] in Global_updatée_7:
    indices = {x: (c.index(x) if x in c else None) for x in range(2, 10)}

    # Vérifie uniquement les indices présents (pas None)
    conditions_ok = all(
        (indices[val] is None) or (indices[val] <= limite)
        for val, limite in limites.items()
    )

    if conditions_ok:
        Global_updatée_8.append([f, c])


1449


In [10]:
print(len(Global))
print(len(Global_updaté))
print(len(Global_updatée_bis))
print(len(Global_updatée_2))
print(len(Global_updatée_3))
print(len(Global_updatée_5))
print(len(Global_updatée_6))
print(len(Global_updatée_7))
print(len(Global_updatée_8))

172024
19536
4884
4559
4559
2117
12702
1449
259


In [11]:
fixes = [
    [ -1, -1, -1, -1, 5, -1, -1, -1, -1],
    [ -1, -1, -1, 4, -1, -1, -1, -1, -1],
    [ -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [ 0, -1, -1, -1, -1, -1, -1, -1, -1],
    [ -1, -1, -1, -1,  1, -1, -1, -1, -1],
    [ -1, -1, -1, -1, -1, -1, 0, 0, 0],
    [ -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [ -1, -1, 0, -1, -1,  8, -1, -1, -1],
    [ -1, -1, 0, -1,  9, -1, -1, -1, -1]
]

from itertools import combinations, product

def verifier_matrices_valides(matrice, matrice_fixe):
    cibles = {n: n for n in range(1, 10)}
    
    positions = {n: [] for n in range(1, 10)}        # Positions modifiables pour chaque chiffre
    positions_fixes = {n: set() for n in range(1, 10)}   # Positions fixées à chaque chiffre
    positions_zero_fixes = set()                      # Positions fixées à 0

    nrows = len(matrice)
    ncols = len(matrice[0])

    for i in range(nrows):
        for j in range(ncols):
            val = matrice[i][j]
            fix_val = matrice_fixe[i][j]

            if fix_val == 0:
                # Case fixée à 0, jamais remplie
                positions_zero_fixes.add((i, j))
            elif fix_val in cibles:
                # Case fixée à un chiffre 1..9
                positions_fixes[fix_val].add((i, j))
            elif fix_val == -1:
                # Case libre
                if val in cibles:
                    positions[val].append((i, j))
            else:
                # Valeur inattendue, on ignore (ou on considère libre)
                if val in cibles:
                    positions[val].append((i, j))

    # Pour chaque chiffre, générer toutes les combinaisons possibles des positions restantes
    for n in range(1, 10):
        fixe = positions_fixes[n]
        dispo = positions[n]
        reste = cibles[n] - len(fixe)
        if reste < 0:
            print((f"Trop de {n} dans les positions fixes ({len(fixe)} > {cibles[n]})"))
            return False
        if len(dispo) < reste:
            print((f"Pas assez de {n} disponibles pour compléter (dispo={len(dispo)}, reste={reste})"))
            return False
    return True

Global_updatée_9 = []

for [f,c] in Global_updatée_8:
    if verifier_matrices_valides(construire_carre_emboite(f, c), fixes):
        Global_updatée_9.append([f,c])

print(len(Global_updatée_9))


def generer_toutes_matrices_valides(matrice, matrice_fixe):
    cibles = range(1, 4)

    nrows, ncols = len(matrice), len(matrice[0])

    # Préparer les containers
    positions_fixes = {n: set() for n in cibles}
    positions_libres = {n: [] for n in cibles}
    positions_zero_fixes = set()

    # Analyse de la matrice fixe
    for i in range(nrows):
        for j in range(ncols):
            fix_val = matrice_fixe[i][j]
            val = matrice[i][j]

            if fix_val == 0:
                positions_zero_fixes.add((i, j))
            elif fix_val in cibles:
                positions_fixes[fix_val].add((i, j))
            elif fix_val == -1 and val in cibles:
                positions_libres[val].append((i, j))
            # Autres valeurs ignorées

    # Générateurs de combinaisons pour chaque chiffre
    def combinaisons_chiffre(n):
        fixe = positions_fixes[n]
        dispo = positions_libres[n]
        quota = n - len(fixe)

        if quota < 0:
            raise ValueError(f"Trop de {n} fixés ({len(fixe)} > {n})")
        if len(dispo) < quota:
            raise ValueError(f"Pas assez de {n} dispos (besoin de {quota}, mais seulement {len(dispo)})")

        # Génère paresseusement les combinaisons
        for comb in combinations(dispo, quota):
            yield (n, fixe.union(comb))

    # Produit cartésien paresseux de toutes les combinaisons possibles
    try:
        produit_complet = product(*(combinaisons_chiffre(n) for n in cibles))
    except ValueError as e:
        # Stop si erreur de configuration
        return

    # Génération finale des matrices
    for choix_comb in produit_complet:
        matrice_resultat = [[0] * ncols for _ in range(nrows)]

        # Appliquer les chiffres
        for n, positions_a_garder in choix_comb:
            for i, j in positions_a_garder:
                matrice_resultat[i][j] = n

        # Appliquer les 0 forcés
        for i, j in positions_zero_fixes:
            matrice_resultat[i][j] = 0

        yield matrice_resultat

# gen = generer_toutes_matrices_valides(construire_carre_emboite(Global_updatée_8[0][0], Global_updatée_8[0][1]), fixes)

# for i, matrice_resultat in enumerate(gen):
#     print(f"Matrice #{i+1}")
#     for ligne in matrice_resultat:
#         print(" ".join(str(x) if x != 0 else "." for x in ligne))
#     print()

#     if i >= 100000:  # Ne montrer que les 3 premières
#         break

grilles = []
i=0
for [f,c] in Global_updatée_9:
    grilles.append([f,c])
    print(i)
    
    gen = generer_toutes_matrices_valides(construire_carre_emboite(f, c), fixes)
    print("done")
    for grille in gen:
        grilles[i].append(grille)
    i+=1

print(grilles)
print(len(grilles))

Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibles pour compléter (dispo=6, reste=7)
Pas assez de 8 disponibl

In [ ]:
grilles_=[]

def vérification_carré(matrice):
    n=len(matrice)
    sum=0
    for k in range(n-1):
        for p in range(n-1):
            carré=[matrice[k][p],matrice[k+1][p],matrice[k][p+1],matrice[k+1][p+1]]
            if 0 not in carré:
                sum+=1
    if sum==0:
        return matrice
    return "erreur"

for grille in grilles:
    grille_ = [grille[0], grille[1]]
    for i in range(2, len(grille)):
        rempli = grille[i]
        a=vérification_carré(rempli)
        if a!="erreur":
            grille_.append(a)
    if len(grille_) > 1:
        grilles_.append(grille_)

grilles = grilles_
print(len(grilles))

211


In [ ]:
for i, grille in enumerate(grilles):
    enregistrement_des_grilles(grille,i)

TypeError: 'list' object cannot be interpreted as an integer